In [1]:
%matplotlib inline
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

import nltk
import gensim


def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')
    


In [2]:
df_pos = pd.read_table('clickbait_data', header=None, names=['X'])
df_neg = pd.read_table('non_clickbait_data', header=None, names=['X'])


# add some ad-hoc data
df_neg = df_neg.append([{'X': '<3'}]*50, ignore_index = True)
df_neg = df_neg.append([{'X': 'connexion login join rejoignez'}]*50, ignore_index = True)


# combine
df_pos['Y'] = True
df_neg['Y'] = False

df = pd.concat([df_pos, df_neg], ignore_index = True)
df = df.reindex(np.random.permutation(df.index))

In [4]:
regex = re.compile(r'\d')
# df_neg.apply(lambda row: len(regex.search(row['X'])) > 0, axis=0)

def f(row):
    return row[1] and regex.search(row[0]) != None

df['list'] = df.apply(f, axis=1)

In [17]:
df_neg

,X,Y
0,Bill Changing Credit Card Rules Is Sent to Oba...,False
1,"In Hollywood, the Easy-Money Generation Toughe...",False
2,1700 runners still unaccounted for in UK's Lak...,False
3,Yankees Pitchers Trade Fielding Drills for Put...,False
4,Large earthquake rattles Indonesia; Seventh in...,False
5,Coldplay's new album hits stores worldwide thi...,False
6,U.N. Leader Presses Sri Lanka on Speeding Reli...,False
7,2 Somali-Americans Charged With Aiding Terror,False
8,US Highway Administration releases interim rep...,False
9,White House Announces International Meetings t...,False


In [5]:
from sklearn.model_selection import KFold


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier


from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


from sklearn.feature_extraction.text import TfidfVectorizer



def gen_scores(y_true, y_pred, y_pred_proba):
    roc = roc_auc_score(y_true, y_pred_proba)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)

    f1 = f1_score(y_true, y_pred)
    
    return { 'roc': roc, 'precision': precision, 'recall': recall, 'f1': f1 }
   
def process_features(sentence):
#     return re.sub(r'\d+', '0', sentence)
    return sentence
#     ret = ""
    
#     for w in sentence:
#         if w.isdigit() and int(w) < 110:
#             ret += '0'
#         else:
#             ret += w
            
#     return ret
    
size = 30
kf = KFold(n_splits=5, shuffle=True)

for train, test in kf.split(df):
    X_train, X_test, y_train, y_test = df['X'][train], df['X'][test], df['Y'][train], df['Y'][test]
    
    tfv = TfidfVectorizer(min_df=3, max_features=None, strip_accents='unicode', analyzer='word',
                     token_pattern=r'\w{1,}', ngram_range=(1, 3),
                     use_idf=1, smooth_idf=1,
                     sublinear_tf=1, stop_words='english')
    
    # gen features
    X_train = [r for r in X_train.apply(lambda sentence: process_features(sentence))]
    X_train = tfv.fit_transform(X_train)
    y_train = list(y_train)
    classifier = LogisticRegression()
#     classifier = GradientBoostingClassifier()
    classifier.fit(X_train, y_train)
    
    X_test = [r for r in X_test.apply(lambda sentence: process_features(sentence))]
    X_test = tfv.transform(X_test).toarray()
    y_test = list(y_test)
    y_pred = classifier.predict(X_test)
    y_pred_proba = classifier.predict_proba(X_test)[:, 1]
    
    
    print(gen_scores(y_test, y_pred, y_pred_proba))

{'roc': 0.99124261187728679, 'precision': 0.96206896551724141, 'recall': 0.95015479876160991, 'f1': 0.95607476635514022}
{'roc': 0.99035281978560175, 'precision': 0.96636771300448432, 'recall': 0.93958268452195581, 'f1': 0.95278698878888368}
{'roc': 0.99051836524282266, 'precision': 0.96524590163934432, 'recall': 0.94571153228397042, 'f1': 0.95537887392503651}
{'roc': 0.991301684532925, 'precision': 0.96606851549755302, 'recall': 0.93851030110935019, 'f1': 0.95209003215434074}
{'roc': 0.99022247686376563, 'precision': 0.9684276336355111, 'recall': 0.94164133738601818, 'f1': 0.95484666358452763}


In [4]:
from sklearn.model_selection import KFold


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier


from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


from sklearn.feature_extraction.text import TfidfVectorizer



def gen_scores(y_true, y_pred, y_pred_proba):
    roc = roc_auc_score(y_true, y_pred_proba)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)

    f1 = f1_score(y_true, y_pred)
    
    return { 'roc': roc, 'precision': precision, 'recall': recall, 'f1': f1 }
   
def process_features(sentence):
#     return re.sub(r'\d+', '0', sentence)
    return sentence
#     ret = ""
    
#     for w in sentence:
#         if w.isdigit() and int(w) < 110:
#             ret += '0'
#         else:
#             ret += w
            
#     return ret
    
size = 30
kf = KFold(n_splits=5, shuffle=True)

for train, test in kf.split(df):
    X_train, X_test, y_train, y_test = df['X'][train], df['X'][test], df['list'][train], df['list'][test]
    
    tfv = TfidfVectorizer(min_df=3, max_features=None, strip_accents='unicode', analyzer='word',
                     token_pattern=r'\w{1,}', ngram_range=(1, 3),
                     use_idf=1, smooth_idf=1,
                     sublinear_tf=1, stop_words='english')
    
    # gen features
    X_train = [r for r in X_train.apply(lambda sentence: process_features(sentence))]
    X_train = tfv.fit_transform(X_train)
    y_train = list(y_train)
    classifier = LogisticRegression()
#     classifier = GradientBoostingClassifier()
    classifier.fit(X_train, y_train)
    
    X_test = [r for r in X_test.apply(lambda sentence: process_features(sentence))]
    X_test = tfv.transform(X_test).toarray()
    y_test = list(y_test)
    y_pred = classifier.predict(X_test)
    y_pred_proba = classifier.predict_proba(X_test)[:, 1]
    
    
    print(gen_scores(y_test, y_pred, y_pred_proba))

{'roc': 0.99038705210473443, 'precision': 0.97776012708498805, 'recall': 0.87181303116147313, 'f1': 0.92175215275177835}
{'roc': 0.98760305315662478, 'precision': 0.97676282051282048, 'recall': 0.87258410880458126, 'f1': 0.92173913043478262}
{'roc': 0.99275668832025299, 'precision': 0.9699918896999189, 'recall': 0.87235594456601018, 'f1': 0.91858678955453155}
{'roc': 0.99268153038229545, 'precision': 0.97751937984496129, 'recall': 0.8708563535911602, 'f1': 0.92111029948867795}
{'roc': 0.99396155907785877, 'precision': 0.9761354888375674, 'recall': 0.88733379986004202, 'f1': 0.92961876832844581}


In [359]:
from sklearn.model_selection import KFold


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier


from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


from sklearn.feature_extraction.text import TfidfVectorizer

def gen_sentence_features(w2v, sentence):
    features = np.array([0.0]*size)
    for w in sentence:
        if w in w2v:
             features += w2v[w]
    
    return features/len(sentence.split(' '))


def gen_scores(y_true, y_pred, y_pred_proba):
    roc = roc_auc_score(y_true, y_pred_proba)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)

    f1 = f1_score(y_true, y_pred)
    
    return { 'roc': roc, 'precision': precision, 'recall': recall, 'f1': f1 }
   
def process_features(sentence):
#     return re.sub(r'\d+', '0', sentence)
    return sentence
#     ret = ""
    
#     for w in sentence:
#         if w.isdigit() and int(w) < 110:
#             ret += '0'
#         else:
#             ret += w
            
#     return ret
    
size = 30
kf = KFold(n_splits=2, shuffle=True)

index = 0
def get_index():
    index += 1
    return index - 1

for train, test in kf.split(df):
    X_train, X_test, y_train, y_test = df['X'][train], df['X'][test], df['Y'][train], df['Y'][test]
    
    X_train = [LabeledSentence(words=r.split(' '), labels = [get_index()]) for r in X_train]
    model = gensim.models.Doc2Vec(X_train, size=size)
    w2v = dict(zip(model.wv.index2word, model.wv.syn0))
    
    # gen features
    X_train = [r for r in X_train.apply(lambda sentence: gen_sentence_features(w2v, process_features(sentence)))]
    y_train = list(y_train)
    classifier = LogisticRegression()
#     classifier = GradientBoostingClassifier()
    classifier.fit(X_train, y_train)
    
    X_test = [r for r in X_test.apply(lambda sentence: gen_sentence_features(w2v, process_features(sentence)))]
    y_test = list(y_test)
    y_pred = classifier.predict(X_test)
    y_pred_proba = classifier.predict_proba(X_test)[:, 1]
    
    
    print(gen_scores(y_test, y_pred, y_pred_proba))
    

NameError: name 'LabeledSentence' is not defined

In [265]:
{'roc': 0.81284205049237268, 'precision': 0.7266265060240964, 'recall': 0.75718769617074699, 'f1': 0.7415923762680604}
{'roc': 0.81494818875165942, 'precision': 0.73140742509251522, 'recall': 0.76263380632312672, 'f1': 0.7466942904149656}

'0 ab'

In [194]:
X = [r for r in X_train[0:2]]
X

[array([ 0.62771392, -0.37692767, -0.05795803, -0.51460409,  0.56070054]),
 array([ 0.5270079 , -0.18355932, -0.14096526, -0.44264327,  0.32411522])]

In [195]:
lr = LinearRegression()
# X = np.array([np.array([1, 2]), np.array([3, 4])])
y = [1, 2]
X
lr.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

# Save Model

In [6]:
tfv = TfidfVectorizer(min_df=3, max_features=None, strip_accents='unicode', analyzer='word',
                     token_pattern=r'\w{1,}', ngram_range=(1, 3),
                     use_idf=1, smooth_idf=1,
                     sublinear_tf=1, stop_words='english')
    
# gen features
X = [r for r in df['X'].apply(lambda sentence: process_features(sentence))]
X = tfv.fit_transform(X)
y_clickbait = list(df['Y'])
y_listicle = list(df['list'])
classifier_clickbait = LogisticRegression()
classifier_listicle = LogisticRegression()

#     classifier = GradientBoostingClassifier()
classifier_clickbait.fit(X, y_clickbait)
classifier_listicle.fit(X, y_listicle)

with open('classifier.p', 'wb') as f:
    pickle.dump((classifier_clickbait, classifier_listicle, tfv), f)

# print(gen_scores(y_test, y_pred, y_pred_proba))